In [0]:
%tensorflow_version 2.x
import tensorflow
import tensorflow.keras as keras
import tensorflow.keras.utils
from tensorflow.keras.layers import LSTM, Input, Dense, TimeDistributed, Masking, Bidirectional, Concatenate
from tensorflow.keras.models import Model

In [0]:
tensorflow.__version__

'2.1.0'

# General

In [0]:
alphabet = list("0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'.,;:!?-+*()[]&/ \"#") # %:;&#
alphabet.insert(0, chr(0)) # '\x00' character, i.e., ord(0) to label concatenate
alphabet.insert(1, '\t') # start of sequence
alphabet.insert(2, '\n') # end of sequence

# Model

In [0]:
timesteps = None
input_features = 3
encoder_space = 512
decoder_space = encoder_space * 2
decoder_timesteps = 300

In [0]:
# Encoder
encoder_input = Input(shape=(timesteps, input_features), name='encoder_input')
masking_layer =  Masking(mask_value=0., input_shape=(timesteps, input_features), name='masking_encoder')(encoder_input)
encoder_output, forward_h, forward_c, backward_h, backward_c= Bidirectional(LSTM(encoder_space, return_state=True), name="encoder")(masking_layer)

# merge states
state_h = Concatenate(name="merge_h")([forward_h, backward_h])
state_c = Concatenate(name="merge_c")([forward_c, backward_c])
encoder_states = [state_h, state_c]

#Decoder
decoder_input = Input(shape=(timesteps, len(alphabet)), name='decoder_input')
decoder_masking = Masking(mask_value=0., input_shape=(timesteps, len(alphabet)), name="masking_decoder")(decoder_input)
decoder_outputs, _, _ = LSTM(decoder_space, return_sequences=True, return_state=True, name="decoder")(decoder_masking, initial_state=encoder_states)
decoder_dense = TimeDistributed(Dense(len(alphabet), activation='softmax'), name="dense_decoder")(decoder_outputs)

In [0]:
model = Model([encoder_input, decoder_input], decoder_dense)
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Preprocessing

## Load Data

In [0]:
import numpy as np

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
training_dataset = np.load('/content/drive/My Drive/deepwriting/deepwriting_training.npz', allow_pickle=True)
validation_dataset = np.load('/content/drive/My Drive/deepwriting/deepwriting_validation.npz', allow_pickle=True)

## Generate Functions

In [0]:
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

In [0]:
def pad_to_length(sequences, max_len, two_dimensional=True):
    padded_sequence = []
    for seq in sequences:
        pad_len = max_len - len(seq)
        if two_dimensional:
            padded_seq = np.pad(seq, [(0, pad_len), (0, 0)], mode='constant', constant_values=0)
        else:
            padded_seq = np.pad(seq, (0, pad_len), mode='constant', constant_values=0)
            padded_seq = np.expand_dims(padded_seq, axis=1)
        padded_sequence.append(padded_seq)
    # check whether all lists have actually the same length
    assert len(list(filter(lambda x: x != max_len, [len(seq) for seq in padded_sequence]))) == 0
    return np.array(padded_sequence)

In [0]:
def create_encoder_data(dataset):
  max_len_encoder = len(max(dataset, key=len))
  encoder_input_data = pad_to_length(dataset, max_len_encoder)
  return encoder_input_data

In [0]:
def create_decoder_data(dataset, alphabet):
  # Encoder for char labels
  label_encoder = LabelEncoder()
  label_encoder = label_encoder.fit(alphabet)

  # Prepare texts
  # \t as starting point
  # \n as ending point
  texts = ['\t' + text + '\n' for text in dataset]

  # skip the '\n' for input
  # skip the '\t' for output
  decoder_input_texts  = [text[:-1] for text in texts]
  decoder_output_texts = [text[1:]  for text in texts]

  # Encode the texts
  decoder_input_text_encoded = [
                               label_encoder.transform([char for char in text])
                               for text in decoder_input_texts
                              ]

  decoder_output_text_encoded = [
                               label_encoder.transform([char for char in text])
                               for text in decoder_output_texts
                              ] 

  # Find max length for padding
  max_len_decoder = len(max(decoder_input_texts, key=len))

  # Pad texts
  decoder_input_text_padded = pad_to_length(decoder_input_text_encoded, max_len_decoder, False)
  decoder_output_text_padded = pad_to_length(decoder_output_text_encoded, max_len_decoder, False)

  # transform to categorical
  decoder_input_data = to_categorical(decoder_input_text_padded, num_classes=len(alphabet))
  decoder_output_data = decoder_output_text_padded

  return decoder_input_data, decoder_output_data

# Training texts 

In [0]:
training_encoder_input_data = create_encoder_data(training_dataset['strokes'])

In [0]:
training_decoder_input_data, training_decoder_output_data = create_decoder_data(training_dataset['texts'], alphabet)

## Validation Texts

In [0]:
validation_encoder_input_data = create_encoder_data(validation_dataset['strokes'])

In [0]:
validation_decoder_input_data, validation_decoder_output_data = create_decoder_data(validation_dataset['texts'], alphabet)

# Training

In [0]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

In [0]:
monitor = 'val_accuracy'
mode='max'

modelCheckpoint = ModelCheckpoint('./best_model_seq2seq_512.hdf5', monitor=monitor, save_best_only='True', mode=mode)
earlyStopping = EarlyStopping(monitor=monitor, patience=10, mode=mode)
reduceLROnPlateau = ReduceLROnPlateau(monitor=monitor, factor=0.1, patience=3, mode=mode)

In [0]:
model.fit(
    [training_encoder_input_data, training_decoder_input_data], 
    training_decoder_output_data,
    batch_size=256,
    epochs=100 ,
    callbacks=[modelCheckpoint, earlyStopping, reduceLROnPlateau],
    validation_data=(
        [
         validation_encoder_input_data,
         validation_decoder_input_data
        ]
        , validation_decoder_output_data
    )
    )

Train on 34577 samples, validate on 705 samples
Epoch 1/100
34577/34577 [==============================] - 72s 2ms/sample - loss: 1.9929 - accuracy: 0.5517 - val_loss: 1.6666 - val_accuracy: 0.5570
Epoch 2/100
34577/34577 [==============================] - 59s 2ms/sample - loss: 1.3872 - accuracy: 0.6315 - val_loss: 1.3339 - val_accuracy: 0.6338
Epoch 3/100
34577/34577 [==============================] - 59s 2ms/sample - loss: 1.1367 - accuracy: 0.6753 - val_loss: 1.1634 - val_accuracy: 0.6637
Epoch 4/100
34577/34577 [==============================] - 59s 2ms/sample - loss: 1.0091 - accuracy: 0.7055 - val_loss: 1.0669 - val_accuracy: 0.6875
Epoch 5/100
34577/34577 [==============================] - 59s 2ms/sample - loss: 0.9085 - accuracy: 0.7317 - val_loss: 0.9759 - val_accuracy: 0.7104
Epoch 6/100
34577/34577 [==============================] - 59s 2ms/sample - loss: 0.8131 - accuracy: 0.7590 - val_loss: 0.9029 - val_accuracy: 0.7291
Epoch 7/100
34577/34577 [===========================